In [102]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [103]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]

In [104]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [105]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [106]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [107]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c
0,2021-06-11T06:00:00.000000000Z,1.21881,1.21930,1.21814,1.21834
1,2021-06-11T07:00:00.000000000Z,1.21834,1.21919,1.21748,1.21830
2,2021-06-11T08:00:00.000000000Z,1.21830,1.21836,1.21639,1.21658
3,2021-06-11T09:00:00.000000000Z,1.21658,1.21672,1.21526,1.21542
4,2021-06-11T10:00:00.000000000Z,1.21542,1.21610,1.21490,1.21514
5,2021-06-11T11:00:00.000000000Z,1.21512,1.21518,1.21297,1.21324
6,2021-06-11T12:00:00.000000000Z,1.21324,1.21379,1.21224,1.21288
7,2021-06-11T13:00:00.000000000Z,1.21290,1.21304,1.21114,1.21213
8,2021-06-11T14:00:00.000000000Z,1.21214,1.21271,1.21006,1.21094
9,2021-06-11T15:00:00.000000000Z,1.21092,1.21148,1.20929,1.21000


In [108]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [109]:
df_ma.dropna(inplace=True)

In [125]:
df_ma.reset_index(drop=True, inplace=True)

In [126]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2021-06-15T21:00:00.000000000Z,1.21266,1.21286,1.21253,1.21282,1.212456,1.212118,0.000338,NaN,False
1,2021-06-15T22:00:00.000000000Z,1.21280,1.21288,1.21261,1.21266,1.212429,1.212030,0.000399,0.000338,False
2,2021-06-15T23:00:00.000000000Z,1.21264,1.21269,1.21236,1.21247,1.212311,1.211938,0.000372,0.000399,False
3,2021-06-16T00:00:00.000000000Z,1.21248,1.21270,1.21225,1.21228,1.212262,1.211871,0.000391,0.000372,False
4,2021-06-16T01:00:00.000000000Z,1.21230,1.21238,1.21206,1.21222,1.212256,1.211821,0.000434,0.000391,False


In [127]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [128]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [129]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [130]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [131]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [132]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
21,2021-06-16T18:00:00.000000000Z,1.21139,1.21139,1.20084,1.20178,1.211601,1.211958,-0.000356,0.000168,True
96,2021-06-21T21:00:00.000000000Z,1.19187,1.19192,1.19174,1.19178,1.190358,1.190194,0.000164,-0.000272,True
176,2021-06-25T05:00:00.000000000Z,1.19374,1.19429,1.19350,1.19408,1.193263,1.193351,-0.000088,0.000030,True
178,2021-06-25T07:00:00.000000000Z,1.19454,1.19508,1.19376,1.19480,1.193518,1.193482,0.000035,-0.000040,True
200,2021-06-28T05:00:00.000000000Z,1.19228,1.19286,1.19220,1.19258,1.193386,1.193556,-0.000170,0.000063,True


In [136]:
df_plot = df_ma.iloc[0:50].copy()

In [137]:
df_plot.shape

(50, 10)

In [138]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1),opacity=1,
    increasing_fillcolor="#24a06b",
    decreasing_fillcolor="#cc2e3c",
    increasing_line_color="#2ec886",
    decreasing_line_color="#ff3a4c"
))
for ma in ma_list:
    col=f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], line=dict(width=2),line_shape="spline", name=col))
fig.update_layout(width=1000,height=300,
margin=dict(l=10,r=10,b=10,t=10),
font=dict(size=10,color="#e1e1e1"),
paper_bgcolor="#1e1e1e",
plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True,
)
fig.show()